# Gathering energies from iterative runs for other embeddings

Notebook that maps a Hamiltonian defined through truncated and penalization embeddings on both the Pegasus and the Zephyr topology, runs it through iterative method and stores energy levels.

Legenda:<br>
<span style="color:cyan">**TO DO (Th)**</span>: notes to myself, additional theory to add for the thesis presentation. <br>
<span style="color:orange">**TO DO (IF)**</span>: notes to myself, a non-necessary idea to implement. <br>
<span style="color:red">**TO DO (Crit)**</span>: problem or doubt to discuss and fix. <br>

## **IMPORTANT**
This notebook is not fully run because I ran out of time on D-Wave QPUs. I'll try to fin a way to run it or I'll just wait january refill.

## Setup

In [1]:
# import
import my_lib as lib 
import article_lib as article

import examples_anf as anf

import numpy as np
import pandas as pd 
from sympy import symbols
import matplotlib.pyplot as plp
from dwave.inspector import show

From `2_qubits_comparison.ipynb` we know that the Zephyr prototype can only support up to $6$ variables for now, so we will only implement such cases.

In [2]:
bits = 6
x = symbols(" ".join((f"x{i}" for i in range(1, bits+1))))

if bits == 4:
    p, sol = anf.example_4_anf(*x)
elif bits == 6:
    p, sol = anf.example_6_anf(*x)
else:
    raise Exception(f'Example with {bits}>6 bits cannot be implemented on Zephyr topology.')

## 

## Truncated Embedding

### Building the Hamiltonian

In [3]:
# symbolic def
truncated = article.truncated_embedding(bits, p, x)
trunc_H = truncated.create_hamiltonian()
trunc_sym = truncated.get_symbols()

# QUBO model
d_art = article.dwave_annealing(trunc_H, bits, trunc_sym)
trunc_H_qubo, trunc_qubo_offset = d_art.symbolic_to_dwave(trunc_H, d_art.get_symbol_num(trunc_sym))

### Pegasus

In [4]:
# class def
trunc_runner = lib.dwave_runners(trunc_H_qubo, trunc_qubo_offset, 
                                 bits, topology='Pegasus', 
                                 chosen_chainstrength='Article')

# iterative call to save energies
p_solution, p_timing_info, p_physical_qubits, final_it = trunc_runner.iterative()
p_energies_hist = trunc_runner.energies_hist

Building the BQM model.
Running on Pegasus Topology.
Number of variables: 90
You chose the Article chainstrength: 28.797385620915033.
Finished running the experiment!
Energy of best sample at iteration 0: 5.0
Best sample: [1 1 1 1 1 1]
Fixing ancillae...
Fixed 3 qubits:
15 : 1
17 : 1
33 : 1

------------------------------------------------------------------------

Number of variables: 87
You chose the Article chainstrength: 28.75233644859813.
Finished running the experiment!
Energy of best sample at iteration 1: 6.0
Best sample: [1 1 1 1 1 1]
Fixing ancillae...
Fixed 6 qubits:
15 : 1
17 : 1
33 : 1
24 : 0
44 : 0
53 : 0

------------------------------------------------------------------------

Number of variables: 84
You chose the Article chainstrength: 28.711111111111112.
Finished running the experiment!
Energy of best sample at iteration 2: 10.0
Best sample: [1 1 1 1 0 1]
Fixing ancillae...
Fixed 10 qubits:
15 : 1
17 : 1
33 : 1
24 : 0
44 : 0
53 : 0
38 : 1
45 : 0
69 : 1
78 : 0

--------

### Zephyr

In [7]:
# class def
trunc_runner = lib.dwave_runners(trunc_H_qubo, trunc_qubo_offset, 
                                 bits, topology='Zephyr', 
                                 chosen_chainstrength='Article')

# iterative call to save energies
z_solution, z_timing_info, z_physical_qubits, final_it = trunc_runner.iterative()
z_energies_hist = trunc_runner.energies_hist

Building the BQM model.
Running on Zephyr Topology.
Number of variables: 90
You chose the Article chainstrength: 28.797385620915033.
Finished running the experiment!
Energy of best sample at iteration 0: 4.0
Best sample: [0 0 1 1 1 1]
Fixing ancillae...
Fixed 21 qubits:
15 : 1
16 : 1
17 : 1
19 : 1
25 : 1
40 : 1
41 : 1
42 : 1
43 : 0
45 : 1
51 : 1
52 : 1
53 : 1
54 : 1
62 : 1
70 : 1
76 : 1
77 : 1
86 : 0
87 : 1
88 : 1

------------------------------------------------------------------------

Number of variables: 69
You chose the Article chainstrength: 30.218181818181815.
Finished running the experiment!
Energy of best sample at iteration 1: -5.0
Best sample: [1 0 1 1 1 1]
Fixing ancillae...
Fixed 29 qubits:
15 : 1
16 : 1
17 : 0
19 : 0
25 : 1
40 : 0
41 : 1
42 : 1
43 : 0
45 : 1
51 : 1
52 : 0
53 : 1
54 : 1
62 : 1
70 : 1
76 : 1
77 : 1
86 : 0
87 : 1
88 : 1
18 : 0
20 : 0
29 : 1
31 : 1
35 : 0
37 : 0
39 : 0
60 : 1

------------------------------------------------------------------------

Number of

SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project DEV

### Comparing

In [ ]:
for i in range (5):
    plp.hist([p_energies_hist[i], z_energies_hist[i]], histtype='stepfilled', 
             color=['blue', 'green'], label=['Pegasus', 'Zephyr'])
    plp.legend()
    plp.show()

<span style="color:red">**TO DO (Crit)**</span>: I think that this kind of histogram is not a useful data representation, given that the sampling is different for different runs (even on the same topology). How can we better represent the difference bettween topologies?

## Penalization Embedding

### Building the Hamiltonian

In [ ]:
# symbolic def
penalization = article.penalization_embedding(bits, p, x)
pen_H = penalization.create_hamiltonian()
pen_sym = penalization.get_symbols()

# QUBO model
d_art = article.dwave_annealing(pen_H, bits, pen_sym)
pen_H_qubo, pen_qubo_offset = d_art.symbolic_to_dwave(pen_H, d_art.get_symbol_num(pen_sym))

### Pegasus

In [ ]:
# class def
pen_runner = lib.dwave_runners(pen_H_qubo, pen_qubo_offset, 
                                 bits, topology='Pegasus', 
                                 chosen_chainstrength='Article')

# iterative call to save energies
p_solution, p_timing_info, p_physical_qubits, final_it = pen_runner.iterative()
p_energies_hist = pen_runner.energies_hist

### Zephyr

In [ ]:
# class def
pen_runner = lib.dwave_runners(pen_H_qubo, pen_qubo_offset, 
                                 bits, topology='Zephyr', 
                                 chosen_chainstrength='Article')

# iterative call to save energies
z_solution, z_timing_info, z_physical_qubits, final_it = pen_runner.iterative()
z_energies_hist = pen_runner.energies_hist

### Comparing

In [ ]:
for i in range (5):
    plp.hist([p_energies_hist[i], z_energies_hist[i]], histtype='stepfilled', 
             color=['blue', 'green'], label=['Pegasus', 'Zephyr'])
    plp.legend()
    plp.show()